# Install required packages


In [ ]:
from shared import *

## Get Posts and see if they are enriched

In [ ]:
cursor.execute("SELECT ID, post_title, post_content FROM workdifferent_wp_0ksss.wmqzcV0Sj_posts WHERE post_type = 'post' AND post_status = 'publish' ORDER BY post_date desc")
posts = cursor.fetchall()
for post_id, post_title, post_content in posts:
    cursor.execute("SELECT * FROM workdifferent_wp_0ksss.wmqzcV0Sj_post_enrichments WHERE post_id = %s;", (post_id,))
    existing_summary = cursor.fetchone()
    
    if not existing_summary:
        enriched_data = enrich_post(post_title, post_content)
        cursor.execute("""
        INSERT INTO workdifferent_wp_0ksss.wmqzcV0Sj_post_enrichments (post_id, enrichment_datetime, articlesection, description, keywords, extracted_text, summary)
        VALUES (%s, %s, %s, %s, %s, %s, %s);
        """, (post_id, datetime.now(), enriched_data['articleSection'][:80], enriched_data['description'], enriched_data['keywords'], enriched_data['extractedText'],
              enriched_data['summary']))
        db.commit()
        print(f"Post {post_id} enriched and saved.")
    else:
        print(f"Post {post_id} already enriched, skipping.")
cursor.close()
db.close()